import libraries

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from google.oauth2 import service_account
from googleapiclient.discovery import build
import re
import json


Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Build service

In [ ]:
# Path to the uploaded service account key file in Colab
SERVICE_ACCOUNT_FILE = '/content/drive/MyDrive/Colab Notebooks/Merchant Center API/service_account_key.json'

# Define the required API scope
SCOPES = ['https://www.googleapis.com/auth/content']

# Authenticate using the service account
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
)

# Build the Google Content API client
service = build('content', 'v2.1', credentials=credentials)

# Merchant Center account ID (replace with your actual ID)
merchant_id = '5411908926'


# Examples

In [ ]:

# Example: Get details of the Merchant Center account (non-MCA)
try:
    request = service.accounts().get(merchantId=merchant_id, accountId=merchant_id)
    response = request.execute()
    print("Account details:")
    print(json.dumps(response, indent=4))
except Exception as e:
    print(f"An error occurred: {e}")



In [ ]:
# List products in the Merchant Center account
try:
    request = service.products().list(merchantId=merchant_id)
    response = request.execute()
    print("Product list:")
    print(json.dumps(response, indent=4))
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
# Example: Get details of the Merchant Center account (non-MCA)
try:
    # Use the same merchantId and accountId for non-MCA accounts
    request = service.accounts().get(merchantId=merchant_id, accountId=merchant_id)
    response = request.execute()
    print("Account details:")
    print(json.dumps(response, indent=4))
except Exception as e:
    print(f"An error occurred: {e}")



In [ ]:
# Get shipping settings for the Merchant Center account
try:
    request = service.shippingsettings().get(merchantId=merchant_id, accountId=merchant_id)
    response = request.execute()
    print("Shipping settings:")
    print(json.dumps(response, indent=4))
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
try:
    request = service.accountstatuses().list(merchantId=merchant_id)
    response = request.execute()
    print("Account status:")
    print(response)
except Exception as e:
    print(f"An error occurred: {e}")


In [ ]:
product_id = 'online:en:US:09780062441720'
product_id2 = '09798886633368'


try:
    # Retrieve product data
    request = service.products().get(merchantId=merchant_id, productId=product_id)
    response = request.execute()
    print("Product details:")
    print(json.dumps(response, indent=4))
except Exception as e:
    print(f"An error occurred: {e}")


# Scraping Publicaiton Pages (front end) from a US IP and updating the pricing for single issue product

1. Prepare your CSV (e.g., merchant_center_products.csv) with the required columns.
2. Update CSV_FILE_PATH if it’s different.
3. Run script

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from google.oauth2 import service_account
from googleapiclient.discovery import build
import re


# Path to service account key (update as needed)
SERVICE_ACCOUNT_FILE = '/content/drive/MyDrive/Colab Notebooks/Merchant Center API/service_account_key.json'

# API scope
SCOPES = ['https://www.googleapis.com/auth/content']

# Authenticate using the service account
credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES
)

# Build the Google Content API client
service = build('content', 'v2.1', credentials=credentials)

# Merchant Center account ID (update if needed)
MERCHANT_ID = '5411908926'

# Read CSV with product data
csv_file_path = '/content/products.csv'  # Update path if needed
df = pd.read_csv(csv_file_path)

# Headers for web scraping
HEADERS = {
    "User-Agent": "DuckDuckBot/1.0; (+http://duckduckgo.com/duckduckbot.html)",
    "X-Forwarded-For": "34.85.12.34"  # Simulated US IP
}

# Needs feed label, language, ID
def build_product_id(row):
    """Construct the correct product ID format"""
    return f"online:{row['language']}:{row['feed label']}:{row['id']}"

def get_product_info(product_id):
    """Fetch product details from Merchant Center API"""
    try:
        request = service.products().get(merchantId=MERCHANT_ID, productId=product_id)
        product_data = request.execute()
        return product_data
    except Exception as e:
        print(f"Error fetching product {product_id}: {e}")
        return None

def extract_price(url):
    """Scrape the correct price from the product page and extract a valid float."""
    try:
        response = requests.get(url, headers=HEADERS)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the price element (update selector if needed)
        price_element = soup.select_one("[data-testid='singleIssueProduct'] .btn-title")
        if price_element:
            price_text = price_element.text.strip()

            # Extract numbers and decimals (handles both '.' and ',')
            price_numbers = re.findall(r'\d+[\.,]?\d*', price_text)
            if price_numbers:
                # Replace comma with dot for float conversion (handles '1,99' → '1.99')
                clean_price = price_numbers[0].replace(",", ".")
                return float(clean_price)

        print(f"Skipping {url}: No valid price found.")
        return None
    except Exception as e:
        print(f"Error scraping {url}: {e}")
        return None

def update_product_price(product_id, new_price):
    """Update the product price in Merchant Center"""
    try:
        product_data = get_product_info(product_id)
        if not product_data:
            return

        updated_product = {
            "price": {
                "value": str(new_price),
                "currency": "USD"
            }
        }

        request = service.products().update(
            merchantId=MERCHANT_ID,
            productId=product_id,
            body=updated_product
        )
        response = request.execute()
        print(f"Updated {product_id} to ${new_price}")
        return response
    except Exception as e:
        print(f"Error updating product {product_id}: {e}")

# Process each product
for _, row in df.iterrows():
    product_id = build_product_id(row)
    product_info = get_product_info(product_id)

    if product_info and 'link' in product_info:
        product_url = product_info['link']
        correct_price = extract_price(product_url)
        if correct_price:
            update_product_price(product_id, correct_price)
        else:
            print(f"Skipping update for {product_id}: Price not found.")
    else:
        print(f"Skipping {product_id}: No product link found.")

print("✅ Price updates completed.")


# Scraping Publicaiton Pages (schema) from a US IP and updating the images for single issue product

1. Prepare your CSV (e.g., merchant_center_products.csv) with the required columns.
2. Update CSV_FILE_PATH if it’s different.
3. Run script



In [ ]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Google Merchant Center API Setup
SERVICE_ACCOUNT_FILE = "/content/drive/MyDrive/Colab Notebooks/Merchant Center API/service_account_key.json"
SCOPES = ["https://www.googleapis.com/auth/content"]
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build("content", "v2.1", credentials=credentials)

# Your Merchant Center ID
MERCHANT_ID = "5411908926"

# Headers for scraping
HEADERS = {
    "User-Agent": "DuckDuckBot/1.0; (+http://duckduckgo.com/duckduckbot.html)"
}

# CSV file with products
CSV_FILE_PATH = "/content/11.csv"  # Update this path as needed

# Your provided functions
def build_product_id(row):
    """Construct the correct product ID format"""
    return f"online:{row['language']}:{row['feed label']}:{row['id']}"

def get_product_info(product_id):
    """Fetch product details from Merchant Center API"""
    try:
        request = service.products().get(merchantId=MERCHANT_ID, productId=product_id)
        product_data = request.execute()
        return product_data
    except Exception as e:
        print(f"Error fetching product {product_id}: {e}")
        return None

# Extract schema.org JSON-LD, selecting only Product type
def extract_schema(url):
    print(f"Step 1: Scraping URL: {url}")
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, "html.parser", from_encoding="utf-8")
        script_tags = soup.find_all("script", type="application/ld+json")
        if not script_tags:
            print("Step 2: No JSON-LD scripts found")
            return None

        for i, script in enumerate(script_tags, 1):
            try:
                script_text = script.string.encode().decode('utf-8')
                data = json.loads(script_text)
                print(f"Step 2: Found schema {i}: {json.dumps(data, indent=2, ensure_ascii=False)}")
                if data.get("@type") == "Product":
                    print(f"Step 2: Selected Product schema {i}")
                    return data
            except json.JSONDecodeError:
                print(f"Step 2: Schema {i} is invalid JSON")
        print("Step 2: No Product schema found among scripts")
        return None
    except Exception as e:
        print(f"Step 2: Could not scrape {url}: {e}")
        return None

# Get the updated image from schema
def get_updated_image(schema):
    if not schema or schema.get("@type") != "Product":
        print("Step 3: No valid Product schema for image update")
        return None
    image = schema.get("image", None)
    if image:
        print(f"Step 3: Updated image found: {image}")
        return image
    print("Step 3: No image found in schema")
    return None

# Update product image in Merchant Center
def update_product_image(product_id, new_image):
    print(f"Step 4: Updating image for product {product_id}")
    try:
        # Fetch current product data
        product_data = get_product_info(product_id)
        if not product_data:
            print(f"Step 4: Skipping update - could not fetch product {product_id}")
            return

        # Update only the imageLink field
        updated_product = {
            "imageLink": new_image
        }
        request = service.products().update(
            merchantId=MERCHANT_ID,
            productId=product_id,
            body=updated_product,
            updateMask="imageLink"  # Only update the imageLink field
        )
        request.execute()
        print(f"Step 5: Successfully updated image for {product_id} to {new_image}")
    except Exception as e:
        print(f"Step 5: Failed to update image for {product_id}: {e}")

# Main function
def main():
    # Read CSV
    df = pd.read_csv(CSV_FILE_PATH)
    print(f"Loaded {len(df)} products from CSV")

    print("\nStarting product updates...\n")
    for i, row in df.iterrows():
        print(f"--- Processing Product {i+1} of {len(df)} ---")

        # Build product ID
        product_id = build_product_id(row)
        print(f"Product ID: {product_id}")

        # Get URL from CSV (assuming 'link' column exists)
        url = row.get("link")
        if not url:
            print("No URL found in CSV row, skipping")
            continue

        # Scrape schema and get new image
        schema = extract_schema(url)
        new_image = get_updated_image(schema)

        # Update product if new image is found
        if new_image:
            update_product_image(product_id, new_image)
        else:
            print(f"No new image to update for {product_id}")

        print(f"--- Finished Product {i+1} ---\n")

if __name__ == "__main__":
    main()

# Extracting Product Schema and Uploading it to a new data source on GMC (Single Issue)

1. Upload CSV with URLs
2. Change target country, language
3. Change Currency to choose desired schema offer
4. Run *script*



In [ ]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Google Merchant Center API Setup
SERVICE_ACCOUNT_FILE = "/content/drive/MyDrive/Colab Notebooks/Merchant Center API/service_account_key.json"
SCOPES = ["https://www.googleapis.com/auth/content"]
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build("content", "v2.1", credentials=credentials)

# Your Merchant Center ID
merchant_id = "5411908926"

# Headers for scraping
HEADERS = {
    "User-Agent": "DuckDuckBot/1.0; (+http://duckduckgo.com/duckduckbot.html)"
}

# CSV file with URLs
CSV_FILE_PATH = "/content/Newspapers-es.csv"
#df = pd.read_csv(CSV_FILE_PATH, skiprows=1611)


# Target country, language, and publication type
target_country = "ES"
target_language = "es"
publication_type = "Newspaper" # Newspaper or Magazine

# Extract schema.org JSON-LD, selecting only Product type
def extract_schema(url):
    print(f"Step 1: Scraping URL: {url}")
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, "html.parser", from_encoding="utf-8")
        script_tags = soup.find_all("script", type="application/ld+json")
        if not script_tags:
            print("Step 2: No JSON-LD scripts found")
            return None

        for i, script in enumerate(script_tags, 1):
            try:
                script_text = script.string.encode().decode('utf-8')
                data = json.loads(script_text)
                print(f"Step 2: Found schema {i}: {json.dumps(data, indent=2, ensure_ascii=False)}")
                if data.get("@type") == "Product":
                    print(f"Step 2: Selected Product schema {i}")
                    return data
            except json.JSONDecodeError:
                print(f"Step 2: Schema {i} is invalid JSON")
        print("Step 2: No Product schema found among scripts")
        return None
    except Exception as e:
        print(f"Step 2: Could not scrape {url}: {e}")
        return None

# Process schema to get product data (first single-issue offer in target currency)
def process_product_data(schema, target_currency):
    if not schema:
        print("Step 3: No schema provided")
        return None
    print(f"Step 3: Processing schema: {json.dumps(schema, indent=2, ensure_ascii=False)}")

    if schema.get("@type") != "Product":
        print(f"Step 3: Invalid type: {schema.get('@type')}")
        return None

    offers = schema.get("offers", [])
    if isinstance(offers, dict):
        offers = [offers]
    print(f"Step 3: Offers: {offers}")

    # Pick the first single-issue offer in the target currency
    for offer in offers:
        offer_name = offer.get("name", "")
        offer_currency = offer.get("priceCurrency", "")
        offer_type = offer.get("offerType", "")  # Check for subscription
        print(f"Step 3: Checking offer - Name: {offer_name}, Currency: {offer_currency}, Type: {offer_type}")
        if offer_currency == target_currency and offer_type != "Subscription":
            price = offer.get("price")
            # Skip if price is zero
            if str(price) in ["0", "0.0", "0.00"]:
                print(f"Step 3: Skipping product '{offer_name}' - Price is zero")
                return None
            # Proceed with non-zero price
            url_parts = offer.get("url", "").split("/")
            last_segment = url_parts[-1]  # e.g., "the-boston-globe"
            locale_code = ""
            content_types = ["newspapers", "magazines"]
            for i, part in enumerate(url_parts):
                if "pressreader.com" in url_parts[i-1] and part not in content_types and i+1 < len(url_parts) and url_parts[i+1] in content_types:
                    locale_code = part
                    break
            offer_id = f"{locale_code}-{last_segment}" if locale_code else last_segment
            product_data = {
                "offerId": offer_id,
                "title": offer_name,
                "description": schema.get("description", ""),
                "imageLink": schema.get("image", "https://via.placeholder.com/150"),
                "link": offer.get("url"),
                "contentLanguage": target_language,
                "targetCountry": target_country,
                "channel": "online",
                "availability": "in stock",
                "price": {"value": str(price), "currency": offer_currency},
                "customLabel0": f"{publication_type} - Single Issue - {target_language.upper()}"
            }
            print(f"Step 3: Processed product data - Title: {product_data['title']}, Custom Label: {product_data['customLabel0']}")
            return product_data
    print(f"Step 3: No {target_currency} single-issue offer found")
    return None

# Upload product to Google Merchant Center
def upload_product(product_data):
    print(f"Step 4: Starting upload for {product_data['title']}")
    try:
        service.products().insert(merchantId=merchant_id, body=product_data).execute()
        print(f"Step 5: Successfully uploaded: {product_data['title']}")
    except Exception as e:
        print(f"Step 5: Failed to upload {product_data['title']}: {e}")

# Create a feed for the target country
def create_data_source():
    feed_name = "PressReader Feed"
    print("Creating feed...")
    try:
        existing_feeds = service.datafeeds().list(merchantId=merchant_id).execute()
        if any(feed["name"] == feed_name for feed in existing_feeds.get("resources", [])):
            print(f"Feed '{feed_name}' already exists")
            return

        service.datafeeds().insert(
            merchantId=merchant_id,
            body={
                "name": feed_name,
                "contentType": "products",
                "fileName": f"{publication_type} - Single Issue - {target_country}.csv",
                "targets": [
                    {"country": target_country, "language": target_language, "includedDestinations": ["Shopping"]}
                ]
            }
        ).execute()
        print(f"Created feed: {feed_name} targeting {target_country}")
    except Exception as e:
        print(f"Failed to create feed: {e}")

# Main function with required currency parameter
def main(target_currency):
    df = pd.read_csv(CSV_FILE_PATH)
    create_data_source()

    print(f"\nStarting URL processing with target currency: {target_currency}...\n")
    for i, url in enumerate(df["url"], 1):
        print(f"--- Processing URL {i} of {len(df['url'])} ---")
        schema = extract_schema(url)
        product_data = process_product_data(schema, target_currency)
        if product_data:
            upload_product(product_data)
        print(f"--- Finished URL {i} ---\n")

if __name__ == "__main__":
    # Specify your desired currency here (required)
    main("USD")  # Must provide a currency, e.g., "EUR", "GBP", "USD"

# Extracting Product Schema and Uploading it to a new data source on GMC (Subscription)

1. Upload CSV with URLs
2. Change target country, language
3. Change Currency to choose desired schema offer
4. Run *script*

In [ ]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
from google.oauth2 import service_account
from googleapiclient.discovery import build
import time

# Google Merchant Center API Setup
SERVICE_ACCOUNT_FILE = "/content/drive/MyDrive/Colab Notebooks/Merchant Center API/service_account_key.json"
SCOPES = ["https://www.googleapis.com/auth/content"]
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build("content", "v2.1", credentials=credentials)

merchant_id = "5411908926"
HEADERS = {"User-Agent": "DuckDuckBot/1.0; (+http://duckduckgo.com/duckduckbot.html)"}
CSV_FILE_PATH = "/content/Newspapers-es.csv"

# Target country and language
target_country = "ES"
target_language = "es"
publication_type = "Newspaper"

def extract_schema(url):
    print(f"Step 1: Scraping URL: {url}")
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, "html.parser", from_encoding="utf-8")
        script_tags = soup.find_all("script", type="application/ld+json")
        if not script_tags:
            print("Step 2: No JSON-LD scripts found")
            return None

        for i, script in enumerate(script_tags, 1):
            try:
                script_text = script.string.encode().decode('utf-8')
                data = json.loads(script_text)
                print(f"Step 2: Found schema {i}: {json.dumps(data, indent=2, ensure_ascii=False)}")
                if data.get("@type") == "Product":
                    print(f"Step 2: Selected Product schema {i}")
                    return data
            except json.JSONDecodeError:
                print(f"Step 2: Schema {i} is invalid JSON")
        print("Step 2: No Product schema found among scripts")
        return None
    except Exception as e:
        print(f"Step 2: Could not scrape {url}: {e}")
        return None

# Process schema to get product data (first subscription offer in target currency)
def process_product_data(schema, target_currency):
    if not schema:
        print("Step 3: No schema provided")
        return None
    print(f"Step 3: Processing schema: {json.dumps(schema, indent=2, ensure_ascii=False)}")

    if schema.get("@type") != "Product":
        print(f"Step 3: Invalid type: {schema.get('@type')}")
        return None

    offers = schema.get("offers", [])
    if isinstance(offers, dict):
        offers = [offers]
    print(f"Step 3: Offers: {offers}")

    # Pick the first subscription offer in the target currency
    for offer in offers:
        offer_name = offer.get("name", "")
        offer_currency = offer.get("priceCurrency", "")
        offer_type = offer.get("offerType", "")
        print(f"Step 3: Checking offer - Name: {offer_name}, Currency: {offer_currency}, Type: {offer_type}")
        if offer_currency == target_currency and offer_type == "Subscription":
            url_parts = offer.get("url", "").split("/")
            last_segment = url_parts[-1]
            locale_code = ""
            content_types = ["newspapers", "magazines"]
            for i, part in enumerate(url_parts):
                if "pressreader.com" in url_parts[i-1] and part not in content_types and i+1 < len(url_parts) and url_parts[i+1] in content_types:
                    locale_code = part
                    break
            offer_id = f"{locale_code}-{last_segment}-sub" if locale_code else f"{last_segment}-sub"
            product_data = {
                "offerId": offer_id,
                "title": offer_name,
                "description": schema.get("description", ""),
                "imageLink": schema.get("image", "https://via.placeholder.com/150"),
                "link": offer.get("url"),
                "contentLanguage": target_language,
                "targetCountry": target_country,
                "channel": "online",
                "availability": "in stock",
                "price": {"value": str(offer.get("price")), "currency": offer_currency},
                "customLabel0": f"{publication_type} - Subscription - {target_language.upper()}"
            }
            print(f"Step 3: Processed product data - Title: {product_data['title']}")
            return product_data
    print(f"Step 3: No {target_currency} subscription offer found")
    return None

def upload_product(product_data):
    print(f"Step 4: Starting upload for {product_data['title']}")
    try:
        service.products().insert(merchantId=merchant_id, body=product_data).execute()
        print(f"Step 5: Successfully uploaded: {product_data['title']}")
    except Exception as e:
        print(f"Step 5: Failed to upload {product_data['title']}: {e}")

def create_data_source():
    feed_name = "PressReader Feed 2"
    print("Creating feed...")
    try:
        existing_feeds = service.datafeeds().list(merchantId=merchant_id).execute()
        if any(feed["name"] == feed_name for feed in existing_feeds.get("resources", [])):
            print(f"Feed '{feed_name}' already exists")
            return

        service.datafeeds().insert(
            merchantId=merchant_id,
            body={
                "name": feed_name,
                "contentType": "products",
                "fileName": f"{publication_type} - Subscription - {target_country}.csv",
                "targets": [
                    {"country": target_country, "language": target_language, "includedDestinations": ["Shopping"]}
                ]
            }
        ).execute()
        print(f"Created feed: {feed_name} targeting {target_country}")
    except Exception as e:
        print(f"Failed to create feed: {e}")

def main(target_currency):
    df = pd.read_csv(CSV_FILE_PATH)
    create_data_source()

    print(f"\nStarting URL processing with target currency: {target_currency}...\n")
    for i, url in enumerate(df["url"], 1):
        print(f"--- Processing URL {i} of {len(df['url'])} ---")
        schema = extract_schema(url)
        product_data = process_product_data(schema, target_currency)
        if product_data:
            upload_product(product_data)
        print(f"--- Finished URL {i} ---\n")
        time.sleep(1)

if __name__ == "__main__":
    # Specify your desired currency here (required)
    main("USD")  # Must provide a currency, e.g., "EUR", "GBP", "USD"

# Delete Products

1. Upload CSV with required columns
2. Run script

In [ ]:
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Google Merchant Center API Setup
SERVICE_ACCOUNT_FILE = "/content/drive/MyDrive/Colab Notebooks/Merchant Center API/service_account_key.json"
SCOPES = ["https://www.googleapis.com/auth/content"]
MERCHANT_ID = "5411908926"

# CSV file with products to delete
CSV_FILE_PATH = "/content/delete.csv"  # Update this path

# Authenticate and build the service
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build("content", "v2.1", credentials=credentials)

# Your provided functions
def build_product_id(row):
    """Construct the correct product ID format"""
    return f"online:{row['language']}:{row['feed label']}:{row['id']}"

def get_product_info(product_id):
    """Fetch product details from Merchant Center API"""
    try:
        request = service.products().get(merchantId=MERCHANT_ID, productId=product_id)
        product_data = request.execute()
        return product_data
    except Exception as e:
        print(f"Error fetching product {product_id}: {e}")
        return None

# Delete a product by productId
def delete_product(product_id):
    try:
        service.products().delete(merchantId=MERCHANT_ID, productId=product_id).execute()
        print(f"Successfully deleted product: {product_id}")
    except Exception as e:
        print(f"Failed to delete product {product_id}: {e}")

# Main function to process CSV and delete products
def main():
    # Read the CSV
    try:
        df = pd.read_csv(CSV_FILE_PATH)
        print(f"Loaded CSV with {len(df)} products to delete")
    except Exception as e:
        print(f"Failed to load CSV: {e}")
        return

    # Ensure required columns exist
    required_columns = ['language', 'feed label', 'id']
    if not all(col in df.columns for col in required_columns):
        print(f"CSV must contain these columns: {required_columns}")
        return

    # Process each row
    for index, row in df.iterrows():
        product_id = build_product_id(row)
        print(f"\nProcessing row {index + 1} of {len(df)}: {product_id}")

        # Optional: Verify product exists (comment out if not needed)
        product_info = get_product_info(product_id)
        if product_info:
            print(f"Found product: {product_info.get('title', 'Unknown title')}")
        else:
            print(f"Product not found or error, attempting deletion anyway")

        # Delete the product
        delete_product(product_id)

if __name__ == "__main__":
    main()

# Extracting Product Schema and Uploading it to a new data source on GMC (Books)

1. Upload CSV with URLs
2. Change target country, language
3. Change region between CA and US to get CAD or USD Currency in schema
4. Run *script*

In [ ]:
import requests
import json
import pandas as pd
from bs4 import BeautifulSoup
from google.oauth2 import service_account
from googleapiclient.discovery import build

# Google Merchant Center API Setup
SERVICE_ACCOUNT_FILE = "/content/drive/MyDrive/Colab Notebooks/Merchant Center API/service_account_key.json"
SCOPES = ["https://www.googleapis.com/auth/content"]
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build("content", "v2.1", credentials=credentials)

merchant_id = "5411908926"
HEADERS = {"User-Agent": "Mozilla/5.0 (compatible; Naverbot/1.0; +http://naver.com/bot)"}
CSV_FILE_PATH = "/content/Book1.csv"

# Target country, language, and publication type (for Merchant Center)
target_country = "CA"
target_language = "en"
publication_type = "Book"

# US IP for explicit US region requests
US_IP = "104.131.0.1"  # Example US IP (e.g., New York)

def extract_schema(url, region):
    print(f"Step 1: Scraping URL: {url} from region: {region}")
    try:
        # Set headers
        headers = HEADERS.copy()
        headers["Accept-Language"] = "en-US" if region == "US" else "en-CA"
        if region == "US":
            headers["X-Forwarded-For"] = US_IP  # Only set US IP for explicit US region
        print(f"Headers: {headers}")  # Log headers for debugging
        session = requests.Session()  # Use fresh session to avoid cookie persistence
        response = session.get(url, headers=headers, timeout=10)
        response.encoding = 'utf-8'
        print(f"Response URL: {response.url}")  # Log response URL
        soup = BeautifulSoup(response.text, "html.parser", from_encoding="utf-8")
        script_tags = soup.find_all("script", type="application/ld+json")
        if not script_tags:
            print("Step 2: No JSON-LD scripts found")
            return None

        for i, script in enumerate(script_tags, 1):
            try:
                script_text = script.string.encode().decode('utf-8')
                data = json.loads(script_text)
                print(f"Step 2: Found schema {i}: {json.dumps(data, indent=2, ensure_ascii=False)}")
                if data.get("@type") == "Product":
                    print(f"Step 2: Selected Product schema {i}")
                    return data
            except json.JSONDecodeError:
                print(f"Step 2: Schema {i} is invalid JSON")
        print("Step 2: No Product schema found")
        return None
    except Exception as e:
        print(f"Step 2: Could not scrape {url} from {region}: {e}")
        return None

def process_product_data(schema):
    if not schema:
        print("Step 3: No schema provided")
        return None
    print(f"Step 3: Processing schema: {json.dumps(schema, indent=2, ensure_ascii=False)}")

    if schema.get("@type") != "Product":
        print(f"Step 3: Invalid type: {schema.get('@type')}, expected Product")
        return None

    offers = schema.get("offers", {})
    if not isinstance(offers, dict):
        print("Step 3: Invalid offers format for Product schema")
        return None

    offer = offers
    title = schema.get("name", "")
    description = schema.get("description", "No description available")
    image_link = schema.get("image", "https://via.placeholder.com/150")
    gtin = schema.get("gtin13", "")
    brand = schema.get("brand", {}).get("name", "")
    url = offer.get("url", "")

    offer_currency = offer.get("priceCurrency", "")
    price = offer.get("price")
    print(f"Step 3: Processing offer - Name: {title}, Currency: {offer_currency}, Price: {price}")

    if str(price) in ["0", "0.0", "0.00"]:
        print(f"Step 3: Skipping product '{title}' - Price is zero")
        return None

    url_parts = url.split("/")
    last_segment = url_parts[-1]
    locale_code = ""
    content_types = ["books"]
    for i, part in enumerate(url_parts):
        if "pressreader.com" in url_parts[i-1] and part not in content_types and i+1 < len(url_parts) and url_parts[i+1] in content_types:
            locale_code = part
            break
    offer_id = f"{locale_code}-{last_segment}" if locale_code else last_segment
    product_data = {
        "offerId": offer_id,
        "title": title,
        "description": description,
        "imageLink": image_link,
        "link": url,
        "contentLanguage": target_language,
        "targetCountry": target_country,
        "channel": "online",
        "availability": "in stock",
        "price": {"value": str(price), "currency": offer_currency},
        "gtin": gtin,
        "brand": brand,
        "customLabel0": f"{publication_type} - {target_language.upper()}"
    }
    print(f"Step 3: Processed product data - Title: {product_data['title']}, Currency: {offer_currency}")
    return product_data

def upload_product(product_data):
    print(f"Step 4: Starting upload for {product_data['title']}")
    try:
        service.products().insert(merchantId=merchant_id, body=product_data).execute()
        print(f"Step 5: Successfully uploaded: {product_data['title']}")
    except Exception as e:
        print(f"Step 5: Failed to upload {product_data['title']}: {e}")

def main(region):
    df = pd.read_csv(CSV_FILE_PATH)

    print(f"\nStarting URL processing from region: {region}...\n")
    for i, url in enumerate(df["url"], 1):
        print(f"--- Processing URL {i} of {len(df['url'])} ---")
        schema = extract_schema(url, region)
        product_data = process_product_data(schema)
        if product_data:
            upload_product(product_data)
        print(f"--- Finished URL {i} ---\n")

if __name__ == "__main__":
    main("CA")  # Use default Canadian IP for CA